In [1]:
from torch.utils.data import DataLoader
import os
import torch
import sys

from tqdm import tqdm
import numpy as np

from auxilary.utils import *
from dataset import nucleiDataset, nucleiValDataset, nucleiTestDataset
from networkModules.modelUnet3p import UNet_3Plus
from auxilary.lossFunctions import *

In [2]:
config = readConfig('configs/config.sys')

In [3]:
trainPaths = config["trainDataset"]
valPaths = config["valDataset"]
testPaths = config["testDataset"]
train_dataset = nucleiDataset(trainPaths, config)
val_dataset = nucleiValDataset(valPaths, config)
test_dataset = nucleiTestDataset(testPaths, config)
trainLoader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
val_data = DataLoader(val_dataset,batch_size=1,num_workers=4)
test_data = DataLoader(test_dataset,batch_size=1)

In [4]:
model = UNet_3Plus(config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = weightedDiceLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=config["learning_rate"], weight_decay=1e-5)

In [5]:
epochs = 1
for epoch in range(epochs):
    for batch in tqdm(test_data):
        
        image, mask, samencoding = batch


        
        gt = mask.squeeze().float()

        gt = gt.type(torch.float32)

        outputs = model((image.to(device), samencoding.to(device)))
        
        loss = criterion(outputs, gt.to(device))

        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()

        #print(f"Loss: {loss.item()}")


        pass

    print(f"Epoch {epoch+1}/{epochs} completed.")

  0%|          | 0/14 [00:00<?, ?it/s]


torch.Size([1, 256, 16, 16])
torch.Size([1, 256, 64, 64])


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 16 but got size 64 for tensor number 1 in the list.